In [1]:
from texas_gerrymandering_hb4.config import CENSUS_GEO_SHP_FILE
import geopandas as gpd

gdf = gpd.read_file(CENSUS_GEO_SHP_FILE)
gdf.info()

2025-09-13 16:42:29.061 | INFO     | texas_gerrymandering_hb4.config:<module>:11 - PROJ_ROOT path is: /home/aimlexpert/Documents/GitHub/texas-gerrymandering-HB4


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 668757 entries, 0 to 668756
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   STATEFP20   668757 non-null  object  
 1   COUNTYFP20  668757 non-null  object  
 2   TRACTCE20   668757 non-null  object  
 3   BLOCKCE20   668757 non-null  object  
 4   GEOID20     668757 non-null  object  
 5   NAME20      668757 non-null  object  
 6   MTFCC20     668757 non-null  object  
 7   UR20        668757 non-null  object  
 8   UACE20      372842 non-null  object  
 9   UATYPE20    372842 non-null  object  
 10  FUNCSTAT20  668757 non-null  object  
 11  ALAND20     668757 non-null  int64   
 12  AWATER20    668757 non-null  int64   
 13  INTPTLAT20  668757 non-null  object  
 14  INTPTLON20  668757 non-null  object  
 15  HOUSING20   668757 non-null  int64   
 16  POP20       668757 non-null  int64   
 17  geometry    668757 non-null  geometry
dtypes: geometry(1), 

In [2]:
gdf.isnull().sum()

STATEFP20          0
COUNTYFP20         0
TRACTCE20          0
BLOCKCE20          0
GEOID20            0
NAME20             0
MTFCC20            0
UR20               0
UACE20        295915
UATYPE20      295915
FUNCSTAT20         0
ALAND20            0
AWATER20           0
INTPTLAT20         0
INTPTLON20         0
HOUSING20          0
POP20              0
geometry           0
dtype: int64